In [1]:
# %load_ext nb_black
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics as m
import xgboost as xgb
import seaborn as sns
warnings.filterwarnings("ignore")
random_seed = 8
import pickle

In [2]:
temp=['L100700', 'SEX', 'S000300', 'L103000', 'L103100', 'L103300',
       'S000501', 'S000100', 'L101300', 'L190300', 'L100500', 'L101700',
      
        'S000300', 'SEX', 'L103300', 'S000501', 
        'L103100', 'L101300', 'S000502', 'L190300', 'L190400', 'L100700', 'L103000', 'L100800',

        'L190900', 'L190700', 'L104500', 'L190800', 'L190600', 'L190400',
        'L190500', 'L104300', 'L104400', 'S000100', 'L100200',

        'L103300', 'L103100', 'L103000', 'L103200', 'S000300', 'L102900', 'SEX', 
        'L100700', 'L190300', 'L190400', 'L190500', 'L100500',

        'L504700', 'AGE','L101700', 'FIELD_4', 'L100800',
        'S000100', 'L103300', 'L101600', 'L100700', 'S000501', 'FIELD_38',

        'L190300', 'L190500', 'L190400', 'L103100', 'S000300',
        'L100700', 'L103300', 'S000100', 'L103000', 'S000501', 'S000502',

        'L103100', 'L103300', 'SEX', 'S000300', 'L103000', 'L190300', 
        'L100700', 'L190400', 'L190500', 'L100500', 'S000100', 'L101300',

        'L101700', 'SEX', 'L101300', 'FIELD_38', 'S000300', 'L101200', 
        'L100700', 'L190400', 'L103000', 'L103300', 'L190300', 'L100800',
      
       'L103000', 'L100700', 'L101700', 'S000300', 'S000502', 'FIELD_33',
       'L190000', 'L100800', 'L101600', 'S000501',

#         'L100800', 'L104600', 'SEX', 'AGE', 'L101300', 'S000300', 
#         'L103000', 'L103300', 'S000501', 'S000502', 'L101700', 'FIELD_38',
      
#       'L102900', 'L103000', 'L103200', 
      'FIELD_1','FIELD_2','FIELD_33','FIELD_15','CLASS'
     
     ]

In [3]:
# collected form each main diabetic indicator features regressors
cols = np.unique(temp)
print(cols)

['AGE' 'CLASS' 'FIELD_1' 'FIELD_15' 'FIELD_2' 'FIELD_33' 'FIELD_38'
 'FIELD_4' 'L100200' 'L100500' 'L100700' 'L100800' 'L101200' 'L101300'
 'L101600' 'L101700' 'L102900' 'L103000' 'L103100' 'L103200' 'L103300'
 'L104300' 'L104400' 'L104500' 'L190000' 'L190300' 'L190400' 'L190500'
 'L190600' 'L190700' 'L190800' 'L190900' 'L504700' 'S000100' 'S000300'
 'S000501' 'S000502' 'SEX']


In [4]:
xcols=[]
ycols=[]
allxycols=[]

for i in cols:
    xcols.append (i+'_x')
    ycols.append (i+'_y')
    
    allxycols.append (i+'_x')
    allxycols.append (i+'_y')

Read Dataset
====

In [5]:
newdata=pd.read_csv('../DATASET_2019-10-24_light.txt')[['FIELD_1','FIELD_2','CLASS']]

In [6]:
x_original=pd.read_csv('../_XLable_forischemic.txt').drop(columns=['CLASS'])
y_original=pd.read_csv('../_TargetLable_forischemic.txt').drop(columns=['CLASS'])

In [7]:
x_original=pd.merge(x_original,newdata,how='inner', left_on=['FIELD_1','FIELD_2'], right_on=['FIELD_1','FIELD_2'])
x_original.shape

(193691, 409)

In [8]:
y_original=pd.merge(y_original,newdata,how='inner', left_on=['FIELD_1','FIELD_2'], right_on=['FIELD_1','FIELD_2'])
y_original.shape

(193691, 409)

In [9]:
data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)
data.shape

(193730, 817)

Prepare dataset
===

In [10]:
data = data[allxycols].dropna()
print(data.shape)

(64980, 76)


In [11]:
data.groupby('CLASS_y').size()

CLASS_y
0    64510
1      470
dtype: int64

In [12]:
maindata=data.copy()

class0=maindata[maindata.CLASS_y==0]
class1=maindata[maindata.CLASS_y==1]

class0=class0.sample(class1.shape[0],random_state=42)

data=pd.concat([class1,class0])

print(data.shape)

data.groupby(by='CLASS_y').size()

(940, 76)


CLASS_y
0    470
1    470
dtype: int64

In [13]:
excludedvalues=maindata.drop(data.index)#.sample(10000,random_state=42)
xtrain2=excludedvalues[xcols]
ytrain2=excludedvalues[ycols]
print(xtrain2.shape)

(64040, 38)


In [14]:
# x = data[xcols]
# y = data[ycols]

# xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=42, test_size=0.3)


In [15]:

test=pd.concat([data[data.CLASS_y==0].sample(100,random_state=42),
               data[data.CLASS_y==1].sample(100,random_state=42)])           
train=data.drop(test.index)

xtrain=train[xcols] 
xtest=test[xcols] 
ytrain=train[ycols]  
ytest=test[ycols] 


In [16]:
xtrain_original=xtrain.copy()
ytrain_original=ytrain.copy()

xtrain=pd.concat([xtrain,xtrain2])
ytrain=pd.concat([ytrain,ytrain2])

print(xtrain.shape,xtest.shape)

(64780, 38) (200, 38)


In [17]:
xtest.groupby(by='CLASS_x').size()

CLASS_x
0    163
1     37
dtype: int64

Train Regression Models
==========

L100700_x # Uric Acid
====

In [18]:
regr = RandomForestRegressor(
    max_depth=10, random_state=random_seed, verbose=0, n_estimators=500, max_features=8
)
regr.fit(
    xtrain[
        [
            'L100700_x', 'SEX_x', 'S000300_x', 'L103000_x', 'L103100_x', 'L103300_x',
       'S000501_x', 'S000100_x', 'L101300_x', 'L190300_x', 'L100500_x', 'L101700_x'
        ]
    ],
    ytrain[["L100700_y"]],
)

ypred = regr.predict(
    xtest[
        [
            'L100700_x', 'SEX_x', 'S000300_x', 'L103000_x', 'L103100_x', 'L103300_x',
       'S000501_x', 'S000100_x', 'L101300_x', 'L190300_x', 'L100500_x', 'L101700_x'
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L100700_y"]], ypred))

r2_score 0.7822343821091118


In [19]:
pred_L100700 = ypred
pred_L100700.shape


with open('models/ISchemiaModel_nextyear_L100700_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 
    

S000300 #BMI
===

In [20]:
regr = RandomForestRegressor(
    max_depth=10, random_state=random_seed, verbose=0, n_estimators=500, max_features=4
)
regr.fit(
    xtrain[
        [            
      'S000300_x', 'SEX_x', 'L103300_x', 'S000501_x','L103100_x', 'L101300_x', 'S000502_x', 
            'L190300_x', 'L190400_x', 'L100700_x', 'L103000_x', 'L100800_x',
        ]
    ],
    ytrain[["S000300_y"]],
)

ypred = regr.predict(
    xtest[
        [
            'S000300_x', 'SEX_x', 'L103300_x', 'S000501_x','L103100_x', 'L101300_x', 'S000502_x', 
            'L190300_x', 'L190400_x', 'L100700_x', 'L103000_x', 'L100800_x',
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["S000300_y"]], ypred))

r2_score 0.9239080545014529


In [21]:
pred_S000300 = ypred

with open('models/ISchemiaModel_nextyear_S000300_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

L101700 # r-GTP gamma
===

In [22]:
regr = RandomForestRegressor(
    max_depth=8, random_state=random_seed, verbose=0, n_estimators=1000
)
regr.fit(
    xtrain[
        ['L101700_x', 'SEX_x', 'L101300_x', 'FIELD_38_x', 'S000300_x', 'L101200_x', 
        'L100700_x', 'L190400_x', 'L103000_x', 'L103300_x', 'L190300_x', 'L100800_x',
        ]
    ],
    ytrain[["L101700_y"]],
)

ypred = regr.predict(
    xtest[
        ['L101700_x', 'SEX_x', 'L101300_x', 'FIELD_38_x', 'S000300_x', 'L101200_x', 
        'L100700_x', 'L190400_x', 'L103000_x', 'L103300_x', 'L190300_x', 'L100800_x',
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L101700_y"]], ypred))

r2_score 0.7141103024070554


In [23]:
pred_L101700 = ypred
with open('models/ISchemiaModel_nextyear_L101700_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

L100800 # fasting glucose level
====

In [24]:
# regr = RandomForestRegressor(
#     max_depth=10, random_state=random_seed, verbose=0, n_estimators=1000, max_features=6
# )
# regr.fit(
#     xtrain[
#         [
#            'L100800_x', 'L104600_x', 'SEX_x', 'AGE_x', 'L101300_x', 'S000300_x', 
#                        'L103000_x', 'L103300_x', 'S000501_x', 'S000502_x', 'L101700_x', 'FIELD_38_x',
#         ]
#     ],
#     ytrain[["L100800_y"]],
# )

# ypred = regr.predict(
#     xtest[
#         [
#             'L100800_x', 'L104600_x', 'SEX_x', 'AGE_x', 'L101300_x', 'S000300_x', 
#                        'L103000_x', 'L103300_x', 'S000501_x', 'S000502_x', 'L101700_x', 'FIELD_38_x',
#         ]
#     ]
# )

# print("r2_score", m.r2_score(ytest[["L100800_y"]], ypred))

In [25]:
# pred_L100800 = ypred

L1033 # Cardiac risk factor
=======

In [26]:
regr = RandomForestRegressor(
    max_depth=10, random_state=random_seed, verbose=0, n_estimators=1000, max_features=10
)
regr.fit(
    xtrain_original[
        [
           'L103300_x', 'L103100_x', 'L103000_x', 'L103200_x', 'S000300_x', 'L102900_x', 'SEX_x', 
                       'L100700_x', 'L190300_x', 'L190400_x', 'L190500_x', 'L100500_x','AGE_x'
        ]
    ],
    ytrain_original[["L103300_y"]],
)

ypred = regr.predict(
    xtest[
        [
            'L103300_x', 'L103100_x', 'L103000_x', 'L103200_x', 'S000300_x', 'L102900_x', 'SEX_x', 
                       'L100700_x', 'L190300_x', 'L190400_x', 'L190500_x', 'L100500_x','AGE_x'
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L103300_y"]], ypred))

r2_score 0.6411854012416756


In [27]:
pred_L103300 = ypred
with open('models/ISchemiaModel_nextyear_L103300_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

103100 # HDL
===

In [28]:
regr = RandomForestRegressor(
    max_depth=10, random_state=random_seed, verbose=0, n_estimators=2000, max_features=10
)
regr.fit(
    xtrain[
        [
           'L103100_x', 'L103300_x', 'SEX_x', 'S000300_x', 'L103000_x', 'L190300_x', 
        'L100700_x', 'L190400_x', 'L190500_x', 'L100500_x', 'S000100_x', 'L101300_x'
        ]
    ],
    ytrain[["L103100_y"]],
)

ypred = regr.predict(
    xtest[
        [
           'L103100_x', 'L103300_x', 'SEX_x', 'S000300_x', 'L103000_x', 'L190300_x', 
        'L100700_x', 'L190400_x', 'L190500_x', 'L100500_x', 'S000100_x', 'L101300_x'
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L103100_y"]], ypred))

r2_score 0.5532557186716084


In [29]:
pred_L103100 = ypred
with open('models/ISchemiaModel_nextyear_L103100_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

L190900 # 
====

In [30]:
regr = RandomForestRegressor(
    max_depth=11, random_state=random_seed, verbose=0, n_estimators=1000, max_features=2
)
regr.fit(
    xtrain_original[
        [
           'L190900_x', 'L190700_x', 'L104500_x', 'L190800_x', 'L190600_x', 'L190400_x',
        'L190500_x', 'L104300_x', 'L104400_x', 'S000100_x', 'L100200_x',
        ]
    ],
    ytrain_original[["L190900_y"]],
)

ypred = regr.predict(
    xtest[
        [
          'L190900_x', 'L190700_x', 'L104500_x', 'L190800_x', 'L190600_x', 'L190400_x',
        'L190500_x', 'L104300_x', 'L104400_x', 'S000100_x', 'L100200_x',
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L190900_y"]], ypred))

r2_score 0.67256041892092


In [31]:
pred_L190900 = ypred
with open('models/ISchemiaModel_nextyear_L190900_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

L504700 # 
====

In [32]:
regr = RandomForestRegressor(
    max_depth=10, random_state=random_seed, verbose=0, n_estimators=1000, max_features=7
)
regr.fit(
    xtrain[
        [
           'L504700_x', 'AGE_x','L101700_x', 'FIELD_4_x', 'L100800_x',
        'S000100_x', 'L103300_x', 'L101600_x', 'L100700_x', 'S000501_x', 'FIELD_38_x',
        ]
    ],
    ytrain[["L504700_y"]],
)

ypred = regr.predict(
    xtest[
        [
           'L504700_x', 'AGE_x','L101700_x', 'FIELD_4_x', 'L100800_x',
        'S000100_x', 'L103300_x', 'L101600_x', 'L100700_x', 'S000501_x', 'FIELD_38_x',
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L504700_y"]], ypred))


r2_score 0.8510882273439483


In [33]:
pred_L504700 = ypred
with open('models/ISchemiaModel_nextyear_L504700_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

RBC #
===

In [34]:
regr = RandomForestRegressor(
    max_depth=10, random_state=random_seed, verbose=0, n_estimators=1000, max_features=7
)
regr.fit(
    xtrain[
        [
           'L190300_x', 'L190500_x', 'L190400_x', 'L103100_x', 'S000300_x',
        'L100700_x', 'L103300_x', 'S000100_x', 'L103000_x', 'S000501_x', 'S000502_x'
        ]
    ],
    ytrain[["L190300_y"]],
)

ypred = regr.predict(
    xtest[
        [
           'L190300_x', 'L190500_x', 'L190400_x', 'L103100_x', 'S000300_x',
        'L100700_x', 'L103300_x', 'S000100_x', 'L103000_x', 'S000501_x', 'S000502_x'
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L190300_y"]], ypred))


r2_score 0.6958949654496958


In [35]:
pred_L190300 = ypred
with open('models/ISchemiaModel_nextyear_L190300_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

Triglycerides #
===

In [36]:
regr = RandomForestRegressor(
    max_depth=10, random_state=random_seed, verbose=0, n_estimators=1000, max_features=7
)
regr.fit(
    xtrain[
        [
           'L103000_x', 'L100700_x', 'L101700_x', 'S000300_x', 'S000502_x', 'FIELD_33_x',
       'L190000_x', 'L100800_x', 'L101600_x', 'S000501_x'
        ]
    ],
    ytrain[["L103000_y"]],
)

ypred = regr.predict(
    xtest[
        [
           'L103000_x', 'L100700_x', 'L101700_x', 'S000300_x', 'S000502_x', 'FIELD_33_x',
       'L190000_x', 'L100800_x', 'L101600_x', 'S000501_x'
        ]
    ]
)

print("r2_score", m.r2_score(ytest[["L103000_y"]], ypred))


r2_score 0.42707940360689367


In [37]:
pred_L103000_y = ypred
with open('models/ISchemiaModel_nextyear_L103000_RF_regressor', 'wb') as f:
    pickle.dump(regr, f) 

In [38]:
ytest.groupby(by='CLASS_y').size()

CLASS_y
0    100
1    100
dtype: int64

In [39]:
xtrain=xtrain_original
ytrain=ytrain_original

Categorical values
=====

In [40]:
pred_FIELD_15 = ytest.FIELD_15_y

In [41]:
pred_FIELD_38 = ytest.FIELD_38_y

In [42]:
pred_FIELD_33 = ytest.FIELD_33_y

In [43]:
pred_SEX = ytest.SEX_y

In [44]:
pred_AGE = ytest.AGE_y + 1

Combine the predicted values to make dataframe
====

In [45]:
NextYearData = pd.DataFrame()
NextYearData["P_FIELD_1"] = ytest.FIELD_1_y.values
NextYearData["P_FIELD_2"] = ytest.FIELD_2_y.values

NextYearData["P_L100700"] = pred_L100700
NextYearData["P_S000300"] = pred_S000300
NextYearData["P_L101700"] = pred_L101700
NextYearData["P_L103300"] = pred_L103300
NextYearData["P_L103100"] = pred_L103100
NextYearData["P_L190900"] = pred_L190900
NextYearData["P_L504700"] = pred_L504700
NextYearData["P_L190300"] = pred_L190300
NextYearData["P_L103000"] = pred_L103000_y

NextYearData["P_FIELD_15"] = pred_FIELD_15.values
NextYearData["P_FIELD_38"] = pred_FIELD_38.values
NextYearData["P_FIELD_33"] = pred_FIELD_33.values

NextYearData["P_SEX"] = pred_SEX.values
NextYearData["P_AGE"] = pred_AGE.values

NextYearData["CLASS"] = ytest.CLASS_y.values

In [46]:
NextYearData.to_csv("Predicted_NextYearData.txt", sep=",")

==============================================
==========

In [47]:
# select the actual next year values

NextYearData_actualData = pd.DataFrame()
NextYearData_actualData["FIELD_1"] = ytest.FIELD_1_y.values
NextYearData_actualData["FIELD_2"] = ytest.FIELD_2_y.values

NextYearData_actualData["L100700"] = ytest.L100700_y.values
NextYearData_actualData["S000300"] = ytest.S000300_y.values
NextYearData_actualData["L101700"] = ytest.L101700_y.values
NextYearData_actualData["L103300"] = ytest.L103300_y.values
NextYearData_actualData["L103100"] = ytest.L103100_y.values 
NextYearData_actualData["L190900"] = ytest.L190900_y.values 
NextYearData_actualData["L504700"] = ytest.L504700_y.values
NextYearData_actualData["L190300"] = ytest.L190300_y.values 
NextYearData_actualData["L103000"] = ytest.L103000_y.values

NextYearData_actualData["FIELD_15"] = ytest.FIELD_15_y.values
NextYearData_actualData["FIELD_38"] = ytest.FIELD_38_y.values
NextYearData_actualData["FIELD_33"] = ytest.FIELD_33_y.values
NextYearData_actualData["SEX"] = ytest.SEX_y.values
NextYearData_actualData["AGE"] = ytest.AGE_y.values

NextYearData_actualData["CLASS"] = ytest.CLASS_y.values


NextYearData_actualData.to_csv("NextYearData_actualData_test.txt", sep=",")

==========================================================
===

In [48]:
# select the actual this year values

ThisYearData_actualData = pd.DataFrame()
ThisYearData_actualData["FIELD_1"] = xtest.FIELD_1_x.values
ThisYearData_actualData["FIELD_2"] = xtest.FIELD_2_x.values

ThisYearData_actualData["L100700"] = xtest.L100700_x.values
ThisYearData_actualData["S000300"] = xtest.S000300_x.values
ThisYearData_actualData["L101700"] = xtest.L101700_x.values
ThisYearData_actualData["L103300"] = xtest.L103300_x.values
ThisYearData_actualData["L103100"] = xtest.L103100_x.values
ThisYearData_actualData["L190900"] = xtest.L190900_x.values
ThisYearData_actualData["L504700"] = xtest.L504700_x.values
ThisYearData_actualData["L190300"] = xtest.L190300_x.values
ThisYearData_actualData["L103000"] = xtest.L103000_x.values

ThisYearData_actualData["FIELD_15"] = xtest.FIELD_15_x.values
ThisYearData_actualData["FIELD_38"] = xtest.FIELD_38_x.values
ThisYearData_actualData["FIELD_33"] = xtest.FIELD_33_x.values

ThisYearData_actualData["SEX"] = xtest.SEX_x.values
ThisYearData_actualData["AGE"] = xtest.AGE_x.values 

ThisYearData_actualData["CLASS"] = xtest.CLASS_x.values 

ThisYearData_actualData.to_csv("ThisYearData_actualData_test.txt", sep=",")

============================================
=========

In [49]:
# select the actual next year values train

NextYearData_actualData_train = pd.DataFrame()
NextYearData_actualData_train["FIELD_1"] = ytrain.FIELD_1_y.values
NextYearData_actualData_train["FIELD_2"] = ytrain.FIELD_2_y.values

NextYearData_actualData_train["L100700"] = ytrain.L100700_y.values
NextYearData_actualData_train["S000300"] = ytrain.S000300_y.values
NextYearData_actualData_train["L101700"] = ytrain.L101700_y.values
NextYearData_actualData_train["L103300"] = ytrain.L103300_y.values
NextYearData_actualData_train["L103100"] = ytrain.L103100_y.values 
NextYearData_actualData_train["L190900"] = ytrain.L190900_y.values 
NextYearData_actualData_train["L504700"] = ytrain.L504700_y.values
NextYearData_actualData_train["L190300"] = ytrain.L190300_y.values 
NextYearData_actualData_train["L103000"] = ytrain.L103000_y.values


NextYearData_actualData_train["FIELD_15"] = ytrain.FIELD_15_y.values
NextYearData_actualData_train["FIELD_38"] = ytrain.FIELD_38_y.values
NextYearData_actualData_train["FIELD_33"] = ytrain.FIELD_33_y.values

NextYearData_actualData_train["SEX"] = ytrain.SEX_y.values
NextYearData_actualData_train["AGE"] = ytrain.AGE_y.values


NextYearData_actualData_train["CLASS"] = ytrain.CLASS_y.values


NextYearData_actualData_train.to_csv("NextYearData_actualData_train.txt", sep=",")

=================================================
==============

In [50]:
# select the actual this year values train

ThisYearData_actualData_train = pd.DataFrame()
ThisYearData_actualData_train["FIELD_1"] = xtrain.FIELD_1_x.values
ThisYearData_actualData_train["FIELD_2"] = xtrain.FIELD_2_x.values

ThisYearData_actualData_train["L100700"] = xtrain.L100700_x.values
ThisYearData_actualData_train["S000300"] = xtrain.S000300_x.values
ThisYearData_actualData_train["L101700"] = xtrain.L101700_x.values
ThisYearData_actualData_train["L103300"] = xtrain.L103300_x.values
ThisYearData_actualData_train["L103100"] = xtrain.L103100_x.values
ThisYearData_actualData_train["L190900"] = xtrain.L190900_x.values
ThisYearData_actualData_train["L504700"] = xtrain.L504700_x.values
ThisYearData_actualData_train["L190300"] = xtrain.L190300_x.values
ThisYearData_actualData_train["L103000"] = xtrain.L103000_x.values


ThisYearData_actualData_train["FIELD_15"] = xtrain.FIELD_15_x.values
ThisYearData_actualData_train["FIELD_38"] = xtrain.FIELD_38_x.values
ThisYearData_actualData_train["FIELD_33"] = xtrain.FIELD_33_x.values

ThisYearData_actualData_train["SEX"] = xtrain.SEX_x.values
ThisYearData_actualData_train["AGE"] = xtrain.AGE_x.values

ThisYearData_actualData_train["CLASS"] = xtrain.CLASS_x.values

ThisYearData_actualData_train.to_csv("ThisYearData_actualData_train.txt", sep=",")